# Estimating hotspot(s) emission and rotational axis from visibilties. 

Notes:
Inner most stable circular orbit (ISCO), for spin=0 with r_g=2 this is at 3M \
Overleaf notes: https://www.overleaf.com/project/60ff0ece5aa4f90d07f2a417

In [1]:
import sys
sys.path.append('../bhnerf')

import os

import jax
from jax import random
from jax import numpy as jnp
import jax.scipy.ndimage as jnd
import scipy.ndimage as nd

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

import xarray as xr
import flax
from flax.training import train_state
from flax.training import checkpoints
import optax
import numpy as np
import matplotlib.pyplot as plt

import utils, emission_utils, visualization, network_utils, observation_utils
from network_utils import shard

import ehtim as eh
import ehtim.const_def as ehc
from tensorboardX import SummaryWriter
from datetime import datetime
from tqdm.notebook import tqdm
import ipyvolume as ipv
from ipywidgets import interact
import ipywidgets as widgets

import yaml
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import glob, yaml

from ehtim.observing.obs_helpers import ftmatrix
# from jax.config import config
# config.update("jax_debug_nans", True)
%load_ext autoreload
%autoreload 2

2021-11-05 12:54:27.719459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-3se_cqee because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



In [ ]:
# Generate hotspot emission (fancy, 4 sheared + 4 new hotspots)
nt, nx, ny, nz = 128, 64, 64, 64
nt_test = 128
r_isco = 3.0 

phi = np.pi/5               # azimuth angle (ccw from x-axis)
theta = np.pi/2 + np.pi/3   # zenith angle (pi/2 = equatorial plane)

# phi = 0.0
# theta = np.pi/3
orbit_radius = 3.5 
std = .4 * np.ones_like(orbit_radius)
initial_frame_single = emission_utils.generate_hotspots_3d(nx, ny, nz, theta, phi, orbit_radius, std, r_isco, std_clip=np.inf)
rot_axis = np.array([np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)])

nspots = 4
# Generated by np.random.choice(range(0, 128, 5), size=nspots, replace=False) 
# with np.random.seed(12345)
frame_subset_inds = np.array([110,  65,  40, 100]) 
frame_subset_inds = np.array([125,  65, 115,  15]) 
# Sum over rotated positions of n_spots
initial_frame_nspots = emission_utils.generate_orbit_3d(initial_frame_single, nt, 1, rot_axis).isel(
    t=frame_subset_inds).sum('t')

orbit_period = 3.5**(-3./2.) 
velocity_field = lambda r: (1.0 / orbit_period) * r**(-3/2)

# Shear initial hotspots
normalization_factor = 0.03
emission = normalization_factor * emission_utils.generate_orbit_3d(initial_frame_nspots, nt, velocity_field, rot_axis)
emission_test = emission

print('rotation axis: {}'.format(rot_axis))

rotation axis: [-0.70062927 -0.50903696 -0.5       ]


In [4]:
extent = [(float(emission[dim].min()), float(emission[dim].max())) for dim in ('x', 'y', 'z')]
@interact(t=widgets.IntSlider(min=0, max=emission.t.size-1, step=1, value=0))
def plot_vol(t):
    ipv.figure()
    ipv.view(0, -60, distance=2.5)
    ipv.volshow(emission.isel(t=t), extent=extent, memorder='F', level=[0, 0.2, 0.7], opacity=[0, 0.2, 0.3], controls=False)
    ipv.show()

interactive(children=(IntSlider(value=0, description='t', max=127), Output()), _dom_classes=('widget-interact'…

In [3]:
# Load sensor 
sensor = xr.load_dataset('../sensors/a0.00_th1.57_ngeo100_npix4096.nc')
sensor = sensor.where(sensor.r < 5)
r_min = sensor.r.min().data   # Minimum supervision radius
r_max = sensor.r.max().data
sensor = sensor.fillna(0.0)

# Generate image plane movies
image_hs = emission_utils.integrate_rays(emission, sensor).data.reshape(nt, sensor.num_alpha, sensor.num_beta)
image_hs_test = emission_utils.integrate_rays(emission_test, sensor).data.reshape(nt_test, sensor.num_alpha, sensor.num_beta)

# Add background emission
riaf_flux = 2.0
riaf_angles = [1, 18, 36, 54, 72, 90]
riaf_angle = min(riaf_angles, key=lambda x:abs(x-np.rad2deg(theta)))
riaf_path = '../riaf/blackhole-{:02d}-10.fits'.format(riaf_angle)
riaf_emission = eh.image.load_fits(riaf_path)
riaf_emission = riaf_emission.rotate(angle=-np.pi/2)
riaf_emission = riaf_emission.regrid_image(targetfov=40*ehc.RADPERUAS, npix=nx)
riaf_emission.imvec = riaf_emission.imvec * (riaf_flux / riaf_emission.total_flux())

image_plane = image_hs + riaf_emission.imarr()
image_plane_test = image_hs_test + riaf_emission.imarr()

Loading fits image:  ../riaf/blackhole-90-10.fits


In [5]:
%matplotlib widget
image_plane_xr = xr.DataArray(image_plane, dims=['t', 'alpha', 'beta'])
image_plane_xr.utils_visualization.animate(cmap='afmhot')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Image Plane

In [5]:
def intensity_to_nchw(intensity, cmap='viridis', gamma=0.5):
    cm = plt.get_cmap(cmap)
    norm_images = ( (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity)) )**gamma
    nchw_images = np.moveaxis(cm(norm_images)[...,:3], (0, 1, 2, 3), (3, 2, 0, 1))
    return nchw_images

def train_network(sensor, emission_true, emission_test, velocity_field, obs_times, hparams, runname, 
                  log_period=100, save_period=10000, x_res_vis=64, y_res_vis=64, z_res_vis=64):
    
    bg_image = jnp.array(riaf_emission.imarr(), dtype=np.float32)
    tstart = 0.0
    tstop = 1.0

    # Training / testing coordinates
    train_coords = network_utils.get_input_coords(sensor, t_array=np.linspace(0., 1., emission.t.size), batch='t')
    t, x, y, z, d = train_coords.values()
    train_vis = measurements

    test_coords = network_utils.get_input_coords(sensor, t_array=np.linspace(0., 1., emission.t.size), batch='t')
    t_test, x_test, y_test, z_test, d_test = test_coords.values()
    test_vis = measurements_test
    
    # Emission visualization inputs
    t_res_vis = hparams['batchsize']
    emission_extent = [emission_true.x[0], emission_true.x[-1], emission_true.y[0], emission_true.y[-1], emission_true.z[0], emission_true.z[-1]]
    t_vis, x_vis, y_vis, z_vis  = np.meshgrid(np.full(t_res_vis, fill_value=tstart), 
                                              np.linspace(emission_extent[0], emission_extent[1], x_res_vis),
                                              np.linspace(emission_extent[2], emission_extent[3], y_res_vis),
                                              np.linspace(emission_extent[4], emission_extent[5], x_res_vis),
                                              indexing='ij')

    d_vis = np.ones_like(y_vis)                  # meaningless placeholder for emission visualization
    r_min = hparams['r_min']                     # Zero network output where there is no supervision (within black-hole radius)  
    r_max = hparams['r_max']
    
    # Model setup and initialization
    rng = jax.random.PRNGKey(1)
    predictor = network_utils.PREDICT_EMISSION_AND_MLP_ROTAXIS_3D(posenc_deg=hparams['posenc_deg'])
    params = predictor.init(rng, x[:1, ...], y[:1, ...], z[:1, ...], t[:1, ...], velocity_field, tstart, tstop)['params']
    tx = optax.adam(learning_rate=optax.polynomial_schedule(hparams['lr_init'], hparams['lr_final'], 1, hparams['num_iters']))
    state = train_state.TrainState.create(apply_fn=predictor.apply, params=params.unfreeze(), tx=tx)  # TODO(pratul): this unfreeze feels sketchy
    
    if np.isscalar(save_period):
        checkpoint_dir = 'checkpoints/{}'.format(runname)
        state = checkpoints.restore_checkpoint(checkpoint_dir, state)
    init_step = 1 + state.step

    train_pstep = jax.pmap(predictor.train_step, axis_name='batch', in_axes=(None, None, 0, 0, 0, 0, 0, 0, None, None, None, 0, 0, 0, None, 0, 0), static_broadcasted_argnums=(0))
    eval_pstep = jax.pmap(predictor.eval_step, axis_name='batch', in_axes=(None, None, 0, 0, 0, 0, 0, 0, None, None, None, 0, 0, 0, None, 0, 0), static_broadcasted_argnums=(0))
    rand_key = jax.random.split(rng, jax.local_device_count())
    state = flax.jax_utils.replicate(state)

    # TensorBoard logging
    time = datetime.now().strftime('%Y-%m-%d.%H:%M:%S')
    logdir = 'runs/{}.{}'.format(runname, time)
       
    with SummaryWriter(logdir=logdir) as writer:

        # Log ground-truth data   
        %matplotlib inline
        images_true = intensity_to_nchw(emission_true.isel(t=0))

        for i in tqdm(range(init_step, init_step+hparams['num_iters']), desc='iteration'):

            # Testing and Visualization
            if (i == 1) or (i % log_period) == 0:
                batch_inds = np.random.choice(range(x.shape[0]), hparams['batchsize'], replace=False)
                
                loss_test, _, _, rendering_test, _, _ = eval_pstep(
                    velocity_field, i, shard(x[batch_inds, ...]), shard(y[batch_inds, ...]), shard(z[batch_inds, ...]), 
                    shard(d[batch_inds, ...]), shard(t[batch_inds, ...]), shard(uv[batch_inds, ...]),
                    fov, tstart, tstop, shard(test_vis[batch_inds,...]), shard(image_plane_test[batch_inds, ...]), shard(sigma_test[batch_inds, ...]), bg_image, state, rand_key
                )
                writer.add_scalar('log loss/test', np.log10(np.mean(loss_test)), global_step=i)
                
                # Log prediction and estimate
                _, _, emission_vis, _, axis_estimation, _ = eval_pstep(
                    velocity_field, i, shard(x_vis), shard(y_vis), shard(z_vis), shard(d_vis), shard(t_vis),
                    shard(uv[batch_inds, ...]), fov, tstart, tstop, shard(test_vis[batch_inds,...]), shard(image_plane_test[batch_inds, ...]),
                    shard(sigma_test[batch_inds, ...]), bg_image, state, rand_key
                )
                axis_estimation = axis_estimation[0] / np.sqrt(np.dot(axis_estimation[0], axis_estimation[0]))
                emission_vis = np.reshape(emission_vis[0,0], [x_res_vis, y_res_vis, z_res_vis])
                emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 < r_min**2, jnp.zeros_like(emission_vis), emission_vis)
                emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 > r_max**2, jnp.zeros_like(emission_vis), emission_vis)
                
                
                # Log emission and rotation axis
                images = intensity_to_nchw(emission_vis[0])
                writer.add_images('emission/estimate', images, global_step=i)
                writer.add_images('emission/true', images_true, global_step=i)
                emission_mse = float(np.mean((emission.data[0] - emission_vis)**2))
                emission_psnr =  float(10.0 * np.log10(np.max(emission.data[0])**2 / emission_mse))
                writer.add_scalar('emission_mse', emission_mse, global_step=i)
                writer.add_scalar('emission_psnr', emission_psnr, global_step=i)
                writer.add_image('input_bg', bg_image[None, ...], global_step=i)
                writer.add_image('rendering/test', rendering_test[0, 0, None, :, :], global_step=i)
                writer.add_image('rendering/true', image_plane_test[batch_inds[0], None, ...], global_step=i)
                writer.add_image('rendering/diff', np.abs(image_plane_test[batch_inds[0], None, ...] - rendering_test[0, 0, None, :, :]), global_step=i)
                writer.add_scalar('rotation/dot_product', np.dot(rot_axis, axis_estimation), global_step=i)
                writer.add_scalar('rotation/x', axis_estimation[0], global_step=i)
                writer.add_scalar('rotation/y', axis_estimation[1], global_step=i)
                writer.add_scalar('rotation/z', axis_estimation[2], global_step=i)
                
            # Training
            batch_inds = np.random.choice(range(x.shape[0]), hparams['batchsize'], replace=False)
            loss_train, state, _, _, _, rand_key = train_pstep(
                velocity_field, i, shard(x[batch_inds, ...]), shard(y[batch_inds, ...]), shard(z[batch_inds, ...]), 
                shard(d[batch_inds, ...]), shard(t[batch_inds, ...]), shard(uv[batch_inds, ...]), fov, tstart, tstop,
                shard(train_vis[batch_inds, ...]), shard(image_plane[batch_inds, ...]), shard(sigma[batch_inds, ...]), bg_image, state, rand_key
            )
            writer.add_scalar('log loss/train', np.log10(np.mean(loss_train)), global_step=i)
    
            
            if np.isscalar(save_period) and ((i % save_period == 0) or (i == hparams['num_iters'])):
                state_to_save = jax.device_get(jax.tree_map(lambda x: x[0], state))
                checkpoints.save_checkpoint(checkpoint_dir, state_to_save, int(i), keep=5)
            
    return state, emission_vis, axis_estimation

In [6]:
# Training parameters
hparams = {
    'num_iters': 5000,
    'lr_init': 1e-3,
    'lr_final': 1e-5,
    'posenc_deg': 3,
    'batchsize': 9,
    'r_min': r_min,
    'r_max': r_max,
}

runname = 'eht_arrays/{}_nspots{:2.2f}_phi{:2.2f}_theta{:2.2f}'.format('full_image', nspots, phi, theta)
state, emission_vis, rot_axis_est = train_network(
    sensor, emission, emission_test, velocity_field, obs_times, hparams, 
    runname=runname, log_period=100, save_period=1000, x_res_vis=64, y_res_vis=64, z_res_vis=64
)

# save results
save_results = {
    'true_rot_axis': rot_axis.tolist(),
    'est_rot_axis': rot_axis_est.tolist(),
    'rot_axis_dotprod': float(np.dot(rot_axis, rot_axis_est)),
    'emission_mse': float(np.mean((emission_vis[0]-emission.isel(t=0))**2))
}
with open(os.path.join('checkpoints', runname, 'results.yml'), 'w') as outfile:
    yaml.dump(save_results, outfile)

# Save emission
emission.name = 'true'
emission_estimate = emission_utils.generate_orbit_3d(
    xr.DataArray(emission_vis[0], coords=emission.isel(t=0).coords), nt, velocity_field, rot_axis_est)
emission_estimate.name = 'estimated'
emission_ds = xr.merge([emission, emission_estimate])
emission_ds.to_netcdf(os.path.join('checkpoints', runname, 'emission_ds.nc'))

NameError: name 'obs_times' is not defined

## ngEHT / EHT2017

In [8]:
def intensity_to_nchw(intensity, cmap='viridis', gamma=0.5):
    cm = plt.get_cmap(cmap)
    norm_images = ( (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity)) )**gamma
    nchw_images = np.moveaxis(cm(norm_images)[...,:3], (0, 1, 2, 3), (3, 2, 0, 1))
    return nchw_images

def train_network(sensor, emission_true, emission_test, velocity_field, obs_times, hparams, runname, 
                  log_period=100, save_period=10000, x_res_vis=64, y_res_vis=64, z_res_vis=64):
    
    bg_image = jnp.array(riaf_emission.imarr(), dtype=np.float32)
    tstart = obs_times[0]
    tstop = obs_times[-1]

    # Training / testing coordinates
    train_coords = network_utils.get_input_coords(sensor, t_array=obs_times, batch='t')
    t, x, y, z, d = train_coords.values()
    train_vis = measurements

    test_coords = network_utils.get_input_coords(sensor, t_array=obs_times, batch='t')
    test_vis = measurements_test
    t_test, x_test, y_test, z_test, d_test = test_coords.values()

    # Emission visualization inputs
    t_res_vis = hparams['batchsize']
    emission_extent = [emission_true.x[0], emission_true.x[-1], emission_true.y[0], emission_true.y[-1], emission_true.z[0], emission_true.z[-1]]
    t_vis, x_vis, y_vis, z_vis  = np.meshgrid(np.full(t_res_vis, fill_value=tstart), 
                                              np.linspace(emission_extent[0], emission_extent[1], x_res_vis),
                                              np.linspace(emission_extent[2], emission_extent[3], y_res_vis),
                                              np.linspace(emission_extent[4], emission_extent[5], x_res_vis),
                                              indexing='ij')

    d_vis = np.ones_like(y_vis)                  # meaningless placeholder for emission visualization
    r_min = hparams['r_min']                     # Zero network output where there is no supervision (within black-hole radius)  
    r_max = hparams['r_max']
    
    # Model setup and initialization
    rng = jax.random.PRNGKey(1)
    predictor = network_utils.PREDICT_EMISSION_AND_MLP_ROTAXIS_3D_FROM_VIS_W_BG(posenc_deg=hparams['posenc_deg'])
    params = predictor.init(rng, x[:1, ...], y[:1, ...], z[:1, ...], t[:1, ...], velocity_field, tstart, tstop)['params']
    tx = optax.adam(learning_rate=optax.polynomial_schedule(hparams['lr_init'], hparams['lr_final'], 1, hparams['num_iters']))
    state = train_state.TrainState.create(apply_fn=predictor.apply, params=params.unfreeze(), tx=tx)  # TODO(pratul): this unfreeze feels sketchy
    
    if np.isscalar(save_period):
        checkpoint_dir = 'checkpoints/{}'.format(runname)
        state = checkpoints.restore_checkpoint(checkpoint_dir, state)
    init_step = 1 + state.step
    train_pstep = jax.pmap(predictor.train_step, axis_name='batch', in_axes=(None, None, 0, 0, 0, 0, 0, 0, None, None, 0, 0, None, 0, 0), static_broadcasted_argnums=(0))
    eval_pstep = jax.pmap(predictor.eval_step, axis_name='batch', in_axes=(None, None, 0, 0, 0, 0, 0, 0, None, None, 0, 0, None, 0, 0), static_broadcasted_argnums=(0))
    rand_key = jax.random.split(rng, jax.local_device_count())
    state = flax.jax_utils.replicate(state)

    # TensorBoard logging
    time = datetime.now().strftime('%Y-%m-%d.%H:%M:%S')
    logdir = 'runs/{}.{}'.format(runname, time)
       
    with SummaryWriter(logdir=logdir) as writer:

        # Log ground-truth data   
        %matplotlib inline
        images_true = intensity_to_nchw(emission_true.isel(t=0))

        for i in tqdm(range(init_step, init_step+hparams['num_iters']), desc='iteration'):

            # Testing and Visualization
            if (i == 1) or (i % log_period) == 0:
                batch_inds = np.random.choice(range(x.shape[0]), hparams['batchsize'], replace=False)

                loss_test, _, _, rendering_test, _, _ = eval_pstep(
                    velocity_field, i, shard(x[batch_inds, ...]), shard(y[batch_inds, ...]), shard(z[batch_inds, ...]), 
                    shard(d[batch_inds, ...]), shard(t[batch_inds, ...]), shard(ft_mats[batch_inds, ...]),
                    tstart, tstop, shard(test_vis[batch_inds,...]),  shard(sigma_test[batch_inds, ...]), 
                    bg_image, state, rand_key
                )
                writer.add_scalar('log loss/test', np.log10(np.mean(loss_test)), global_step=i)
                
                # Log prediction and estimate
                _, _, emission_vis, _, axis_estimation, _ = eval_pstep(
                    velocity_field, i, shard(x_vis), shard(y_vis), shard(z_vis), shard(d_vis), shard(t_vis),
                    shard(ft_mats[batch_inds, ...]), tstart, tstop, shard(test_vis[batch_inds,...]),
                    shard(sigma_test[batch_inds, ...]), bg_image, state, rand_key
                )
                axis_estimation = axis_estimation[0] / np.sqrt(np.dot(axis_estimation[0], axis_estimation[0]))
                emission_vis = np.reshape(emission_vis[0,0], [x_res_vis, y_res_vis, z_res_vis])
                emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 < r_min**2, jnp.zeros_like(emission_vis), emission_vis)
                emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 > r_max**2, jnp.zeros_like(emission_vis), emission_vis)
                
                
                # Log emission and rotation axis
                images = intensity_to_nchw(emission_vis[0])
                writer.add_images('emission/estimate', images, global_step=i)
                writer.add_images('emission/true', images_true, global_step=i)
                emission_mse = float(np.mean((emission.data[0] - emission_vis)**2))
                emission_psnr =  float(10.0 * np.log10(np.max(emission.data[0])**2 / emission_mse))
                writer.add_scalar('emission_mse', emission_mse, global_step=i)
                writer.add_scalar('emission_psnr', emission_psnr, global_step=i)
                writer.add_image('input_bg', bg_image[None, ...], global_step=i)
                # vmax = np.max(rendering_test[0, 0, None, :, :])
                vmax = 0.05
                writer.add_image('rendering/test', rendering_test[0, 0, None, :, :] / vmax, global_step=i)
                writer.add_image('rendering/true', image_plane_test[batch_inds[0], None, ...] / vmax, global_step=i)
                writer.add_image('rendering/diff', np.abs(image_plane_test[batch_inds[0], None, ...] - rendering_test[0, 0, None, :, :]), global_step=i)
                writer.add_scalar('rotation/dot_product', np.dot(rot_axis, axis_estimation), global_step=i)
                writer.add_scalar('rotation/x', axis_estimation[0], global_step=i)
                writer.add_scalar('rotation/y', axis_estimation[1], global_step=i)
                writer.add_scalar('rotation/z', axis_estimation[2], global_step=i)
                
            # Training
            batch_inds = np.random.choice(range(x.shape[0]), hparams['batchsize'], replace=False)
            loss_train, state, _, _, _, rand_key = train_pstep(
                velocity_field, i, shard(x[batch_inds, ...]), shard(y[batch_inds, ...]), shard(z[batch_inds, ...]), 
                shard(d[batch_inds, ...]), shard(t[batch_inds, ...]), shard(ft_mats[batch_inds, ...]), tstart, tstop,
                shard(train_vis[batch_inds, ...]), shard(sigma[batch_inds, ...]), bg_image, state, rand_key
            )
            writer.add_scalar('log loss/train', np.log10(np.mean(loss_train)), global_step=i)
    
            
            if np.isscalar(save_period) and ((i % save_period == 0) or (i == hparams['num_iters'])):
                state_to_save = jax.device_get(jax.tree_map(lambda x: x[0], state))
                checkpoints.save_checkpoint(checkpoint_dir, state_to_save, int(i), keep=5)
            
    return state, emission_vis, axis_estimation

In [9]:
# EHT params
array_name = 'ngEHT'
fov = 85.0         # field of view in uas 
seed = None        # random seed for EHT measurement noise

hparams = {
    'num_iters': 5000,
    'lr_init': 1e-4,
    'lr_final': 1e-6,
    'posenc_deg': 3,
    'batchsize': 6,
    'r_min': r_min,
    'r_max': r_max,
}

# Load eht array
obs_params = {
    'array': eh.array.load_txt('../eht_arrays/{}.txt'.format(array_name)),
    'nt': 128,      # number of time samples 
    'tstart': 4.0,  # start of observations (UTC hr)
    'tstop': 15.5,  # end of observation (UTC hr)
    'tint': 60.0    # integration time
}
obs_empty = observation_utils.empty_eht_obs(**obs_params)

obs_args = {
    'psize': fov / sensor.num_alpha * ehc.RADPERUAS,
    'ra': obs_empty.ra, 
    'dec': obs_empty.dec,
    'rf': obs_empty.rf, 
    'mjd': obs_empty.mjd
}

times = np.linspace(obs_params['tstart'], obs_params['tstop'], nt)
times_test = np.linspace(obs_params['tstart'], obs_params['tstop'], nt_test)

movie = eh.movie.Movie(image_plane, times, **obs_args)
movie_test = eh.movie.Movie(image_plane_test, times_test, **obs_args)
riaf_emission = eh.image.Image(riaf_emission.imarr(), **obs_args)

# Generate visibility measurements with noise
obs = observation_utils.observe_same(movie, obs_empty, ttype='direct', seed=seed)
obs_test = observation_utils.observe_same(movie_test, obs_empty, ttype='direct', seed=seed)
obs_riaf = riaf_emission.observe_same_nonoise(obs_empty, ttype='direct')

measurements = observation_utils.padded_obs(obs, 'vis', fill_value=0.0)
measurements_test = observation_utils.padded_obs(obs_test, 'vis', fill_value=0.0)

sigma = observation_utils.padded_obs(obs, 'sigma', fill_value=np.inf)
sigma_test = observation_utils.padded_obs(obs, 'sigma', fill_value=np.inf)

uv = np.stack((observation_utils.padded_obs(obs, 'u'), 
               observation_utils.padded_obs(obs, 'v')), axis=2)

ft_mats = np.stack([ftmatrix(movie.psize, movie.xdim, movie.ydim, uv_t, pulse=movie.pulse) for uv_t in uv])
ft_mats = np.nan_to_num(ft_mats, 0.0)
obs_times = np.array([np.mean(obsdata['time'][0]) for obsdata in obs.tlist()])

Producing clean visibilities from movie with direct FT . . . 


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(datalist)


Applying Jones Matrices to data . . . 
Applying Jones Matrices to data . . . 
Adding thermal noise to data . . . 
Applying a priori calibration with estimated Jones matrices . . . 
Producing clean visibilities from movie with direct FT . . . 
Applying Jones Matrices to data . . . 
Applying Jones Matrices to data . . . 
Adding thermal noise to data . . . 
Applying a priori calibration with estimated Jones matrices . . . 
Producing clean visibilities from image with direct FT . . . 


In [11]:
%matplotlib widget
image_plane_xr = xr.DataArray(image_plane, dims=['t', 'alpha', 'beta'])

plt.rcdefaults()
fig, ax = plt.subplots(1,2, figsize=(9.5,4))
anim = image_plane_xr.utils_visualization.animate(ax=ax[0])
obs.plotall('uvdist', 'amp', axis=ax[1], label='total', ebar=False,  legend=True)
obs_riaf.plotall('uvdist', 'amp', axis=ax[1], label='background', ebar=False, color='r', legend=True, marker='^')
#ax[1].set_ylim([0, 2.5])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/scratch/imaging/projects/bh_nerf/bhNeRF/notebooks/../bhnerf/visualization.py:167: MatplotlibDeprecationWarning: Starting from Matplotlib 3.6, colorbar() will steal space from the mappable's axes, rather than from the current axes, to place the colorbar.  To silence this warning, explicitly pass the 'ax' argument to colorbar().
  fig.colorbar(im)


<AxesSubplot:xlabel='$u-v$ Distance', ylabel='Amplitude'>

In [ ]:
runname = 'eht_arrays/{}_nspots{:2.2f}_phi{:2.2f}_theta{:2.2f}_riaf'.format(array_name, nspots, phi, theta)
state, emission_vis, rot_axis_est = train_network(
    sensor, emission, emission_test, velocity_field, obs_times, hparams, 
    runname=runname, log_period=100, save_period=None, x_res_vis=64, y_res_vis=64, z_res_vis=64
)

emission_mse = float(np.mean((emission.data[0] - emission_vis)**2))
emission_psnr =  float(10.0 * np.log10(np.max(emission.data[0])**2 / emission_mse))
        
# save results
save_results = {
    'true_rot_axis': rot_axis.tolist(),
    'est_rot_axis': rot_axis_est.tolist(),
    'rot_axis_dotprod': float(np.dot(rot_axis, rot_axis_est)),
    'emission_mse': emission_mse,
    'emission_psnr': emission_psnr
}
with open(os.path.join('checkpoints', runname, 'results.yml'), 'w') as outfile:
    yaml.dump(save_results, outfile)

# Save emission
emission.name = 'true'
emission_estimate = emission_utils.generate_orbit_3d(
    xr.DataArray(emission_vis[0], coords=emission.isel(t=0).coords), nt, velocity_field, rot_axis_est)
emission_estimate.name = 'estimated'
emission_ds = xr.merge([emission, emission_estimate])
emission_ds.to_netcdf(os.path.join('checkpoints', runname, 'emission_ds.nc'))

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


iteration:   0%|          | 0/5000 [00:00<?, ?it/s]

## Visualize Results

### Volume rendering

In [34]:
# Perspective camera functions
def normalize(x):
    """Normalization helper function."""
    return x / np.linalg.norm(x)

def viewmatrix(lookdir, up, position):
    """Construct lookat view matrix."""
    vec2 = normalize(lookdir)
    vec0 = normalize(np.cross(up, vec2))
    vec1 = normalize(np.cross(vec2, vec0))
    m = np.stack([vec0, vec1, vec2, position], axis=1)
    return m

def generate_hemispherical_orbit(radius, n_frames=120):
    """Calculates a render path which orbits around the z-axis."""
    # Assume that z-axis points up towards approximate camera hemisphere
    render_poses = []

    up = np.array([0., 0., 1.])
    for theta in np.linspace(0., 2. * np.pi, n_frames):
        camorigin = radius * np.array(
            [np.cos(theta), np.sin(theta), 0.])
        render_poses.append(viewmatrix(camorigin, up, camorigin))

    render_poses = np.stack(render_poses, axis=0)
    return render_poses

def generate_elevated_orbit(radius, height, n_frames=120):
    """Calculates a render path which orbits around the z-axis."""
    # Assume that z-axis points up towards approximate camera hemisphere
    render_poses = []
    
    orbit_radius = np.sqrt(radius**2 - height**2)

    up = np.array([0., 0., 1.])
    for theta in np.linspace(0., 2. * np.pi, n_frames):
        camorigin = np.array(
            [orbit_radius * np.cos(theta), orbit_radius * np.sin(theta), height])
        render_poses.append(viewmatrix(camorigin, up, camorigin))

    render_poses = np.stack(render_poses, axis=0)
    return render_poses

def generate_rays(camtoworlds, width, height, focal):
    """Generating rays for all images."""
    x, y = np.meshgrid(  # pylint: disable=unbalanced-tuple-unpacking
        np.arange(width, dtype=np.float32),  # X-Axis (columns)
        np.arange(height, dtype=np.float32),  # Y-Axis (rows)
        indexing='xy')
    camera_dirs = np.stack(
        [(x - width * 0.5 + 0.5) / focal,
         -(y - height * 0.5 + 0.5) / focal, -np.ones_like(x)],
        axis=-1)
    directions = ((camera_dirs[None, ..., None, :] *
                   camtoworlds[:, None, None, :3, :3]).sum(axis=-1))
    origins = np.broadcast_to(camtoworlds[:, None, None, :3, -1],
                              directions.shape)

    return origins, directions

def sample_along_rays(rays_o, rays_d, near, far, num_samples):
    t_vals = jnp.linspace(near, far, num_samples)
    pts = rays_o[:, :, :, None, :] + t_vals[None, None, None, :, None] * rays_d[:, :, :, None, :]
    return pts

def draw_cube(emission, pts):
    facewidth = 10.
    linewidth = 0.1
#     linecolor = jnp.array([1000.0, 1000.0, 1000.0])
    linecolor = jnp.array([0.0, 0.0, 0.0, 1000.0])
    vertices = jnp.array([[-facewidth/2., -facewidth/2., -facewidth/2.],
                        [facewidth/2., -facewidth/2., -facewidth/2.],
                        [-facewidth/2., facewidth/2., -facewidth/2.],
                        [facewidth/2., facewidth/2., -facewidth/2.],
                        [-facewidth/2., -facewidth/2., facewidth/2.],
                        [facewidth/2., -facewidth/2., facewidth/2.],
                        [-facewidth/2., facewidth/2., facewidth/2.],
                        [facewidth/2., facewidth/2., facewidth/2.]])
    dirs = jnp.array([[-1., 0., 0.],
                      [1., 0., 0.],
                      [0., -1., 0.],
                      [0., 1., 0.],
                      [0., 0., -1.],
                      [0., 0., 1.]])
    
    cm = plt.get_cmap('hot')
    emission_cm = cm(emission)
    emission_cm = jnp.clip(emission_cm - 0.05, 0.0, 1.0)
    
    emission_cm = jnp.concatenate([emission_cm[..., :3], emission[..., None] / jnp.amax(emission)], axis=-1)
    
#     emission = jnp.where(jnp.less(jnp.broadcast_to(emission[..., 0, None], emission.shape), 0.1), 
#                          jnp.zeros_like(emission), emission)

    for i in range(vertices.shape[0]):

        for j in range(dirs.shape[0]):
            # Draw line segments from each vertex
            line_seg_pts = vertices[i, None, :] + jnp.linspace(0.0, facewidth, 64)[:, None] * dirs[j, None, :]

            for k in range(line_seg_pts.shape[0]):
                dists = jnp.linalg.norm(pts - jnp.broadcast_to(line_seg_pts[k, None, None, None, :], pts.shape), axis=-1)
                # out = jnp.where(jnp.less_equal(dists, linewidth),
                #                 lineval * jnp.exp(-1. * dists), out)
                emission_cm += linecolor[None, None, None, :] * jnp.exp(-1. * dists / linewidth ** 2)[..., None]
    
    out = jnp.where(jnp.greater(jnp.broadcast_to(jnp.amax(jnp.abs(pts), axis=-1, keepdims=True), 
                                                 emission_cm.shape), 
                                facewidth/2. + linewidth), 
                    jnp.zeros_like(emission_cm), emission_cm)
        
    return out

def alpha_composite(emission, dists, pts, inside_halfwidth=4.5):
    emission = np.clip(emission, 0., 1.)
    color = emission[..., :-1] * dists[0, ..., None]
    alpha = emission[..., -1:] 
    
    inside = np.where(np.less(np.amax(np.abs(pts), axis=-1), inside_halfwidth), 
                      np.ones_like(pts[..., 0]),
                      np.zeros_like(pts[..., 0]))
    
    rendering = np.zeros_like(color[:, :, 0, :])
    acc = np.zeros_like(color[:, :, 0, 0])
    outside_acc = np.zeros_like(color[:, :, 0, 0])
    for i in range(alpha.shape[-2]):
        ind = alpha.shape[-2] - i - 1
        
        # if pixels inside cube, don't alpha composite
        rendering = rendering + inside[..., ind, None] * color[..., ind, :]
        
        # if pixels not inside cube, alpha composite      
        outside_alpha = alpha[..., ind, :] * (1. - inside[..., ind, None])
        rendering = rendering * (1. - outside_alpha) + color[..., ind, :] * outside_alpha 
        
        acc = alpha[..., ind, 0] + (1. - alpha[..., ind, 0]) * acc
        outside_acc = outside_alpha[..., 0] + (1. - outside_alpha[..., 0]) * outside_acc
        
    rendering += np.array([1., 1., 1.])[None, None, :] * (1. - acc[..., None])
    return rendering, outside_acc

In [30]:
sensor = xr.load_dataset('../sensors/a0.00_th1.57_ngeo100_npix4096.nc')
sensor = sensor.where(sensor.r < 5)
r_min = sensor.r.min().data   # Minimum supervision radius
sensor = sensor.fillna(0.0)

phi = 0.0                # azimuth angle (ccw from x-axis)
theta = np.pi/3          # zenith angle (pi/2 = equatorial plane)
rot_axis = np.array([np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)])
orbit_period = 3.5**(-3./2.) 
velocity_field = lambda r: (1.0 / orbit_period) * r**(-3/2)

velocity_std = 0.35
length_scale = 0.01
random_state = 0
radii = np.linspace(1, 10, 100)

nt, nx, ny, nz = 64, 128, 128, 128
nspots = 1
r_isco = 3.0 
orbit_radius = 3.5
std = .4 * np.ones_like(orbit_radius)
orbit_period = orbit_radius**(-3./2.) 
velocity_field = lambda r: (1.0 / orbit_period) * r**(-3/2)
                        
phi = 0.0         
theta = np.pi / 3 
initial_frame = emission_utils.generate_hotspots_3d(nx, ny, nz, theta, phi, orbit_radius, std, r_isco, std_clip=np.inf)
rot_axis = np.array([np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)])

gpr = GPR(kernel=RBF(length_scale=length_scale))
gp_noise = np.squeeze(velocity_std * gpr.sample_y(radii.reshape(-1, 1), n_samples=1, random_state=random_state))
noisy_velocity_field = lambda r: velocity_field(r) + np.interp(r, radii, gp_noise)
emission_gt = emission_utils.generate_orbit_3d(initial_frame, nt, noisy_velocity_field, rot_axis)

nt = 64
nx = 64
ny = 64
nz = 64

x_res_vis = 256
y_res_vis = 256
z_res_vis = 256

hparams = {
    'num_iters': 50000,
    'lr_init': 1e-4,
    'lr_final': 1e-6,
    'lr_axis': 1e-1,
    'posenc_deg': 3,
    'batchsize': 1,
    'r_min': r_min
}

# Training / testing coordinates
train_coords = network_utils.get_input_coords(sensor, t_array=np.linspace(0, 1, nt), batch='t')
t, x, y, z, d = train_coords.values()
t = np.linspace(0, 1, nt)

# Emission visualization inputs
t_res_vis = 1
emission_extent = [-5, 5, -5, 5, -5, 5]
t_vis, x_vis, y_vis, z_vis  = np.meshgrid(0, np.linspace(emission_extent[0], emission_extent[1], x_res_vis),
                                          np.linspace(emission_extent[2], emission_extent[3], y_res_vis),
                                          np.linspace(emission_extent[4], emission_extent[5], x_res_vis),
                                          indexing='ij')

d_vis = np.ones_like(y_vis)                  # meaningless placeholder for emission visualization
uv = np.zeros((1,171,2))
test_vis = np.zeros((1,171))

# Model setup and initialization
rng = jax.random.PRNGKey(1)
predictor = network_utils.PREDICT_EMISSION_AND_MLP_ROTAXIS_3D_FROM_VIS_W_BG_PRATUL(posenc_deg=hparams['posenc_deg'])
params = predictor.init(rng, x[:1, ...], y[:1, ...], z[:1, ...], t[:1, ...], velocity_field, 0, 1)['params']

def flattened_traversal(fn):
    def mask(data):
        flat = flax.traverse_util.flatten_dict(data)
        return flax.traverse_util.unflatten_dict({k: fn(k, v) for k, v in flat.items()})
    return mask

tx = optax.chain(
    optax.masked(optax.adam(learning_rate=hparams['lr_axis']),
                 mask=flattened_traversal(lambda path, _: path[-1] == 'axis')),
    optax.masked(optax.adam(learning_rate=optax.polynomial_schedule(hparams['lr_init'], hparams['lr_final'], 1, hparams['num_iters'])),
                 mask=flattened_traversal(lambda path, _: path[-1] != 'axis')),
)

state = train_state.TrainState.create(apply_fn=predictor.apply, params=params.unfreeze(), tx=tx)  # TODO(pratul): this unfreeze feels sketchy

rng = jax.random.PRNGKey(1)
rand_key = jax.random.split(rng, jax.local_device_count())
eval_pstep = jax.pmap(predictor.eval_step, axis_name='batch', in_axes=(None, None, 0, 0, 0, 0, 0, 0, None, None, None, 0, 0, 0, None, 0, 0), static_broadcasted_argnums=(0))

In [31]:
poses = generate_elevated_orbit(25., 10., n_frames=60)
height = y_res_vis
width = x_res_vis
focal = .5 * width / jnp.tan(.5 * 0.7)
rays_o, rays_d = generate_rays(poses, width, height, focal)
pts = sample_along_rays(rays_o, rays_d, 15., 35., z_res_vis)

i, t = 0, 0.0

x_vis = pts[i:i+1, :, :, :, 0]
y_vis = pts[i:i+1, :, :, :, 1]
z_vis = pts[i:i+1, :, :, :, 2]
t_vis = t * jnp.ones_like(x_vis)
d_vis = jnp.linalg.norm(jnp.concatenate([jnp.diff(pts[i:i+1, ...], axis=3),
                                         jnp.zeros_like(pts[i:i+1, :, :, -1:])], 
                                         axis=3), axis=-1)

In [35]:
# render ground truth
emission_vis = emission_gt.sel(t=t, method='nearest').interp(x=xr.DataArray(x_vis[0]),
                                                             y=xr.DataArray(y_vis[0]),
                                                             z=xr.DataArray(z_vis[0])).fillna(0.0).data
emission_vis_cube = draw_cube(emission_vis, jnp.stack([x_vis[0], y_vis[0], z_vis[0]], axis=-1))
# rendering_gt = jnp.sum(emission_vis_cube * d_vis[0, ..., None], axis=-2)
rendering_gt, _ = alpha_composite(emission_vis_cube, d_vis, pts[i, ...])
rendering_gt = jnp.clip(rendering_gt, 0.0, 1.0)

In [50]:
checkpoint_paths = [
    'checkpoints/v_noise/ngEHT_NO_V_NOISE/checkpoint_5000',
    'checkpoints/v_noise/ngEHT_amp0.05_corrlen0.35_random_state0_phi0.00_theta1.05/checkpoint_5000',
    'checkpoints/v_noise/ngEHT_amp0.35_corrlen0.01_random_state0_phi0.00_theta1.05/checkpoint_5000',
]

rendering = []
for checkpoint_path in checkpoint_paths:
    print('loading checkpoint...')
    t0 = time.time()
    state = checkpoints.restore_checkpoint(checkpoint_path, state)
    state = flax.jax_utils.replicate(state)

    _, _, emission_vis, rendering_vis, axis_est, _ = eval_pstep(
        velocity_field, 0, 
        shard(x_vis), shard(y_vis), shard(z_vis), shard(d_vis), shard(t_vis), shard(uv), 
        100, 0, 1, 
        shard(test_vis), shard(jnp.zeros((1, y_res_vis, x_res_vis))), shard(jnp.ones_like(test_vis)), 
        jnp.zeros((y_res_vis, x_res_vis)),
        state, rand_key
    )

    emission_vis = np.reshape(emission_vis, [t_res_vis, x_res_vis, y_res_vis, z_res_vis])
    emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 >= hparams['r_min']**2, emission_vis, jnp.zeros_like(emission_vis))
    emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 <= sensor.r.max().data**2, emission_vis, jnp.zeros_like(emission_vis))
    emission_vis = emission_vis / emission_vis.max()
    print('loading time [secs]: {}'.format(time.time() - t0))
        
    print('rendering cube visualization...')
    t0 = time.time()
    emission_vis_cube = draw_cube(emission_vis[0], jnp.stack([x_vis[0], y_vis[0], z_vis[0]], axis=-1))
    image, _ = alpha_composite(emission_vis_cube, d_vis, pts[i, ...])
    print('rendering time [secs]: {}'.format(time.time() - t0))
    image = jnp.clip(image, 0.0, 1.0)
    rendering.append(image)

loading checkpoint...


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


loading time [secs]: 27.832905054092407
rendering cube visualization...
rendering time [secs]: 412.94897985458374
loading checkpoint...


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


loading time [secs]: 31.302058696746826
rendering cube visualization...
rendering time [secs]: 629.683527469635


In [57]:
%matplotlib widget
fig, axes = plt.subplots(1,4, figsize=(12,4))
for ax in axes:
    ax.axis('off')
axes[0].imshow(rendering_gt)
axes[1].imshow(rendering[0])
axes[2].imshow(rendering[1])
axes[3].imshow(rendering[2])

axes[0].set_title('Ground truth', fontsize=14)
axes[1].set_title('No noise', fontsize=14)
axes[2].set_title(r'$\sigma=0.35;~\ell=0.01$', fontsize=14)
axes[3].set_title(r'$\sigma=0.05;~\ell=0.35$', fontsize=14)
plt.tight_layout()

plt.savefig('figures/v_noise_recoveries.pdf', transparent=False, bbox_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
up = np.array([-1., 0., 0.])
camorigin = np.array([9, 0, 20.])
render_poses = viewmatrix(camorigin, up, camorigin)
poses = np.stack([render_poses, render_poses], axis=0)

height = y_res_vis
width = x_res_vis
focal = .5 * width / jnp.tan(.5 * 0.7)
rays_o, rays_d = generate_rays(poses, width, height, focal)
pts = sample_along_rays(rays_o, rays_d, 15., 35., z_res_vis)

i = 0
t = 0.0

# render ground truth
x_vis = pts[i:i+1, :, :, :, 0]
y_vis = pts[i:i+1, :, :, :, 1]
z_vis = pts[i:i+1, :, :, :, 2]
t_vis = t * jnp.ones_like(x_vis)
d_vis = jnp.linalg.norm(jnp.concatenate([jnp.diff(pts[i:i+1, ...], axis=3),
                                         jnp.zeros_like(pts[i:i+1, :, :, -1:])], 
                                         axis=3), axis=-1)

emission_vis = emission_gt.sel(t=t, method='nearest').interp(x=xr.DataArray(x_vis[0]),
                                                             y=xr.DataArray(y_vis[0]),
                                                             z=xr.DataArray(z_vis[0])).data
emission_vis_cube = draw_cube(emission_vis, jnp.stack([x_vis[0], y_vis[0], z_vis[0]], axis=-1))
rendering_gt1, _ = alpha_composite(emission_vis_cube, d_vis, pts[i, ...])
rendering_gt1 = jnp.clip(rendering_gt1, 0.0, 1.0)

checkpoint_paths = [
    'checkpoints/v_noise/ngEHT_NO_V_NOISE/checkpoint_5000',
    'checkpoints/v_noise/ngEHT_amp0.05_corrlen0.35_random_state0_phi0.00_theta1.05/checkpoint_5000',
    'checkpoints/v_noise/ngEHT_amp0.35_corrlen0.01_random_state0_phi0.00_theta1.05/checkpoint_5000',
]

rendering1 = []
for checkpoint_path in checkpoint_paths:
    print('loading checkpoint...')
    t0 = time.time()
    state = checkpoints.restore_checkpoint(checkpoint_path, state)
    state = flax.jax_utils.replicate(state)

    _, _, emission_vis, rendering_vis, axis_est, _ = eval_pstep(
        velocity_field, 0, 
        shard(x_vis), shard(y_vis), shard(z_vis), shard(d_vis), shard(t_vis), shard(uv), 
        100, 0, 1, 
        shard(test_vis), shard(jnp.zeros((1, y_res_vis, x_res_vis))), shard(jnp.ones_like(test_vis)), 
        jnp.zeros((y_res_vis, x_res_vis)),
        state, rand_key
    )

    emission_vis = np.reshape(emission_vis, [t_res_vis, x_res_vis, y_res_vis, z_res_vis])
    emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 >= hparams['r_min']**2, emission_vis, jnp.zeros_like(emission_vis))
    emission_vis = jnp.where(x_vis**2 + y_vis**2 + z_vis**2 <= sensor.r.max().data**2, emission_vis, jnp.zeros_like(emission_vis))
    emission_vis = emission_vis / emission_vis.max()
    print('loading time [secs]: {}'.format(time.time() - t0))
    print('rendering cube visualization...')
    t0 = time.time()
    emission_vis_cube = draw_cube(emission_vis[0], jnp.stack([x_vis[0], y_vis[0], z_vis[0]], axis=-1))
    image, _ = alpha_composite(emission_vis_cube, d_vis, pts[i, ...])
    print('rendering time [secs]: {}'.format(time.time() - t0))
    image = jnp.clip(image, 0.0, 1.0)
    rendering1.append(image)

loading checkpoint...


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


loading time [secs]: 29.58119297027588
rendering cube visualization...
rendering time [secs]: 410.01189947128296
loading checkpoint...


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


loading time [secs]: 28.224812507629395
rendering cube visualization...
rendering time [secs]: 708.1207647323608
loading checkpoint...


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


loading time [secs]: 29.213650226593018
rendering cube visualization...
rendering time [secs]: 506.93497252464294


In [62]:
up = np.array([-1., 0., 0.])
camorigin = np.array([9, 0, 20.])
render_poses = viewmatrix(camorigin, up, camorigin)
poses = np.stack([render_poses, render_poses], axis=0)

height = y_res_vis
width = x_res_vis
focal = .5 * width / jnp.tan(.5 * 0.7)
rays_o, rays_d = generate_rays(poses, width, height, focal)
pts = sample_along_rays(rays_o, rays_d, 15., 35., z_res_vis)

i = 0
t = 0.0

# render ground truth
x_vis = pts[i:i+1, :, :, :, 0]
y_vis = pts[i:i+1, :, :, :, 1]
z_vis = pts[i:i+1, :, :, :, 2]
t_vis = t * jnp.ones_like(x_vis)
d_vis = jnp.linalg.norm(jnp.concatenate([jnp.diff(pts[i:i+1, ...], axis=3),
                                         jnp.zeros_like(pts[i:i+1, :, :, -1:])], 
                                         axis=3), axis=-1)

emission_vis = emission_gt.sel(t=t, method='nearest').interp(x=xr.DataArray(x_vis[0]),
                                                             y=xr.DataArray(y_vis[0]),
                                                             z=xr.DataArray(z_vis[0])).fillna(0.0).data
emission_vis_cube = draw_cube(emission_vis, jnp.stack([x_vis[0], y_vis[0], z_vis[0]], axis=-1))
rendering_gt1, _ = alpha_composite(emission_vis_cube, d_vis, pts[i, ...])
rendering_gt1 = jnp.clip(rendering_gt1, 0.0, 1.0)

In [63]:
%matplotlib widget
fig, axes = plt.subplots(1,4, figsize=(12,3))
for ax in axes:
    ax.axis('off')

axes[0].imshow(rendering_gt1)
axes[1].imshow(rendering1[0])
axes[2].imshow(rendering1[1])
axes[3].imshow(rendering1[2])

axes[0].set_title('Ground truth', fontsize=14)
axes[1].set_title('No noise', fontsize=14)
axes[2].set_title(r'$\sigma=0.35;~\ell=0.01$', fontsize=14)
axes[3].set_title(r'$\sigma=0.05;~\ell=0.35$', fontsize=14)
plt.tight_layout()

plt.savefig('figures/v_noise_recoveries_topview.pdf', transparent=False, bbox_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### MSE plot

In [19]:
velocity_stds = np.linspace(0.05, 0.35, 5)
length_scales = np.linspace(0.01, 0.35, 5)
random_states = range(5)

emission_mse = np.empty((5,5,5))

for i, random_state in enumerate(random_states):                               
    for j, velocity_std in enumerate(velocity_stds):
         for k, length_scale in enumerate(length_scales):
                path = 'checkpoints/v_noise/ngEHT_amp{:2.2f}_corrlen{:2.2f}_random_state{}_phi0.00_theta1.05/results.yml'.format(
                    velocity_std, length_scale, random_state
                )
                with open(path) as f:
                    emission_mse[i, j, k] = yaml.load(f, Loader=yaml.FullLoader)['emission_mse']

emission_mse = xr.DataArray(data=emission_mse, dims=['random_state', 'velocity_std', 'length_scale'], 
                            coords={'random_state': random_states, 
                                    'velocity_std': velocity_stds, 
                                    'length_scale': [0.01 , 0.095, 0.18 , 0.265, 0.35 ]})



In [31]:
fig, axes = plt.subplots(1,1, figsize=(4,3.5))
im = emission_mse.mean('random_state').plot(add_colorbar=False)

axes.set_xlabel(r'correlation length $\ell$', fontsize=14)
axes.set_ylabel(r'noise magnitude $\sigma$', fontsize=14)
axes.set_title('Emission MSE', fontsize=16)

divider = make_axes_locatable(axes)
cax = divider.append_axes('right', size='5%', pad=0.08)
cb = fig.colorbar(im, cax=cax)
cb.formatter.set_powerlimits((0, 0))
cb.ax.yaxis.set_offset_position('left') 
plt.tight_layout()
plt.savefig('figures/emission_mse.pdf', transparent=False, bbox_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run a single experiment

In [3]:
# Generate hotspot emission
nt, nx, ny, nz = 128, 64, 64, 64
nt_test = 128
nspots = 1
r_isco = 3.0 

phi = 0.0            # azimuth angle (ccw from x-axis)
theta = np.pi/3      # zenith angle (pi/2 = equatorial plane)
orbit_radius = 3.5 
std = .4 * np.ones_like(orbit_radius)
initial_frame = emission_utils.generate_hotspots_3d(nx, ny, nz, theta, phi, orbit_radius, std, r_isco, std_clip=np.inf)
rot_axis = np.array([np.cos(theta)*np.cos(phi), np.cos(theta)*np.sin(phi), -np.sin(theta)])

orbit_period = 3.5**(-3./2.) 
velocity_field = lambda r: (1.0 / orbit_period) * r**(-3/2)

# Sample noisy velocity field
# radii = np.linspace(1, 10, 100)
# velocity_std = 0.1
# length_scale = 0.1
# random_state = 0
# gpr = GPR(kernel=RBF(length_scale=length_scale))
# gp_noise = np.squeeze(velocity_std * gpr.sample_y(radii.reshape(-1, 1), n_samples=1, random_state=random_state))
# noisy_velocity_field = lambda r: velocity_field(r) + np.interp(r, radii, gp_noise)
# if np.any(noisy_velocity_field(radii) < 0):
#     raise AttributeError('Negative noisy velocity, reduce velocity_std or try different random state')
# 
# emission = emission_utils.generate_orbit_3d(initial_frame, nt, noisy_velocity_field, rot_axis)
# emission_test = emission_utils.generate_orbit_3d(initial_frame, nt_test, noisy_velocity_field, rot_axis)

emission = emission_utils.generate_orbit_3d(initial_frame, nt, velocity_field, rot_axis)
emission_test = emission_utils.generate_orbit_3d(initial_frame, nt_test, velocity_field, rot_axis)

normalization_factor = 0.03
emission *= normalization_factor
emission_test *= normalization_factor

print('rotation axis: {}'.format(rot_axis))

rotation axis: [ 0.5        0.        -0.8660254]


In [4]:
extent = [(float(emission[dim].min()), float(emission[dim].max())) for dim in ('x', 'y', 'z')]
@interact(t=widgets.IntSlider(min=0, max=emission.t.size-1, step=1, value=0))
def plot_vol(t):
    ipv.figure()
    ipv.view(0, -60, distance=2.5)
    ipv.volshow(emission.isel(t=t), extent=extent, memorder='F', level=[0, 0.2, 0.7], opacity=[0, 0.2, 0.3], controls=False)
    ipv.show()

interactive(children=(IntSlider(value=0, description='t', max=127), Output()), _dom_classes=('widget-interact'…

In [4]:
# Generate image plane fluxes
sensor = xr.load_dataset('../sensors/a0.00_th1.57_ngeo100_npix4096.nc')
sensor = sensor.where(sensor.r < 5)
r_min = sensor.r.min().data   # Minimum supervision radius
sensor = sensor.fillna(0.0)

image_plane = emission_utils.integrate_rays(emission, sensor).data.reshape(nt, sensor.num_alpha, sensor.num_beta)
image_plane_test = emission_utils.integrate_rays(emission_test, sensor).data.reshape(nt_test, sensor.num_alpha, sensor.num_beta)

In [5]:
fov = 85.0         # field of view in uas 
array_name = 'ngEHT'

obs_params = {
    'array': eh.array.load_txt('../eht_arrays/{}.txt'.format(array_name)),
    'nt': 128,      # number of time samples 
    'tstart': 4.0,  # start of observations (UTC hr)
    'tstop': 15.5,  # end of observation (UTC hr)
    'tint': 60.0    # integration time
}
obs_empty = observation_utils.empty_eht_obs(**obs_params)
obs_args = {
    'psize': fov / sensor.num_alpha * ehc.RADPERUAS,
    'ra': obs_empty.ra, 
    'dec': obs_empty.dec,
    'rf': obs_empty.rf, 
    'mjd': obs_empty.mjd
}

%matplotlib widget
observation_utils.plot_uv_coverage(obs_empty)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(datalist)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# Generate visibility measurements with NO noise
# times = np.linspace(obs_params['tstart'], obs_params['tstop'], nt)
# times_test = np.linspace(obs_params['tstart'], obs_params['tstop'], nt_test)
# 
# movie = eh.movie.Movie(image_plane, times, **obs_args)
# movie_test = eh.movie.Movie(image_plane, times_test, **obs_args)
# 
# obs = movie.observe_same_nonoise(obs_empty, ttype='fast')
# obs_test = movie_test.observe_same_nonoise(obs_empty, ttype='fast')
# 
# measurements = observation_utils.padded_obs(obs, 'vis', fill_value=0.0)
# measurements_test = observation_utils.padded_obs(obs_test, 'vis', fill_value=0.0)
# sigma = np.ones_like(measurements, dtype=np.float32)
# sigma_test = np.ones_like(measurements_test, dtype=np.float32)
# 
# uv = np.stack((observation_utils.padded_obs(obs, 'u'), 
#                observation_utils.padded_obs(obs, 'v')), axis=2)
# obs_times = np.array([np.mean(obsdata['time'][0]) for obsdata in obs.tlist()])
# 
# riaf_emission = eh.image.Image(np.zeros_like(image_plane[0]), **obs_args)

Producing clean visibilities from movie with fast FT . . . 
Producing clean visibilities from movie with fast FT . . . 


In [6]:
# Generate visibility measurements with noise
times = np.linspace(obs_params['tstart'], obs_params['tstop'], nt)
times_test = np.linspace(obs_params['tstart'], obs_params['tstop'], nt_test)

movie = eh.movie.Movie(image_plane, times, **obs_args)
movie_test = eh.movie.Movie(image_plane, times_test, **obs_args)
riaf_emission = eh.image.Image(np.zeros_like(image_plane[0]), **obs_args)

seed = 6
obs = observation_utils.observe_same(movie, obs_empty, ttype='fast', seed=seed)
obs_test = observation_utils.observe_same(movie_test, obs_empty, ttype='fast', seed=seed)
obs_riaf = observation_utils.observe_same(riaf_emission, obs_empty, ttype='fast', seed=seed)

measurements = observation_utils.padded_obs(obs, 'vis', fill_value=0.0)
measurements_test = observation_utils.padded_obs(obs_test, 'vis', fill_value=0.0)

sigma = observation_utils.padded_obs(obs, 'sigma', fill_value=np.inf)
sigma_test = observation_utils.padded_obs(obs, 'sigma', fill_value=np.inf)

uv = np.stack((observation_utils.padded_obs(obs, 'u'), 
               observation_utils.padded_obs(obs, 'v')), axis=2)

obs_times = np.array([np.mean(obsdata['time'][0]) for obsdata in obs.tlist()])

Producing clean visibilities from movie with fast FT . . . 
Applying Jones Matrices to data . . . 
Applying Jones Matrices to data . . . 
Adding thermal noise to data . . . 
Applying a priori calibration with estimated Jones matrices . . . 
Producing clean visibilities from movie with fast FT . . . 
Applying Jones Matrices to data . . . 
Applying Jones Matrices to data . . . 
Adding thermal noise to data . . . 
Applying a priori calibration with estimated Jones matrices . . . 
Producing clean visibilities from image with fast FT . . . 
Applying Jones Matrices to data . . . 
Adding thermal noise to data . . . 
Applying a priori calibration with estimated Jones matrices . . . 


In [7]:
# Visualize radiance measurements 
%matplotlib widget
plt.rcdefaults()
fig, ax = plt.subplots(1,2, figsize=(9.5,4))
image_plane_visualization = xr.DataArray(image_plane, dims=['t', 'alpha', 'beta'])
anim = image_plane_visualization.utils_visualization.animate(ax=ax[0])
obs.plotall('uvdist', 'amp', axis=ax[1], label='total', ebar=False,  legend=True)
outpath = 'gifs/{}.measurements.fov{:2.1f}.nspots{:d}.theta_{:2.0f}.phi_{:2.0f}_riaf.gif'.format(
    array_name, fov, nspots, np.rad2deg(theta), np.rad2deg(phi))
#anim.save(outpath, writer='imagemagick', fps=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/scratch/imaging/projects/bh_nerf/bhNeRF/notebooks/../bhnerf/visualization.py:167: MatplotlibDeprecationWarning: Starting from Matplotlib 3.6, colorbar() will steal space from the mappable's axes, rather than from the current axes, to place the colorbar.  To silence this warning, explicitly pass the 'ax' argument to colorbar().
  fig.colorbar(im)


In [7]:
# Training parameters
hparams = {
    'num_iters': 5000,
    'lr_init': 1e-4,
    'lr_final': 1e-6,
    'lr_axis': 1e-2,
    'posenc_deg': 3,
    'batchsize': 6,
    'r_min': r_min
}

runname = 'v_noise/ngEHT_NO_V_NOISE'
state, emission_vis, rot_axis_est = train_network(
    sensor, emission, emission_test, velocity_field, obs_times, hparams, 
    runname=runname, log_period=100, save_period=1000, x_res_vis=64, y_res_vis=64, z_res_vis=64
)
emission_estimate = emission_utils.generate_orbit_3d(
    xr.DataArray(emission_vis[0], coords=initial_frame.coords), nt, velocity_field, rot_axis_est)

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:390: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/jax/lib/xla_bridge.py:377: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


iteration:   0%|          | 0/5000 [00:00<?, ?it/s]

In [190]:
extent = [(float(initial_frame[dim].min()), float(initial_frame[dim].max())) for dim in initial_frame.dims]
@interact(t=widgets.IntSlider(min=0, max=emission.t.size-1, step=1, value=0))
def plot_vol(t):
    ipv.figure()
    ipv.view(0, -60, distance=2.5)
    ipv.volshow(emission_estimate.isel(t=t), extent=extent, memorder='F', level=[0, 0.2, 0.7], opacity=[0, 0.2, 0.3], controls=False)
    ipv.show()

interactive(children=(IntSlider(value=0, description='t', max=63), Output()), _dom_classes=('widget-interact',…

In [29]:
# Save measurement prediction
plt.rcdefaults()
image_prediction = emission_utils.integrate_rays(emission_estimate, sensor)
image_prediction = xr.DataArray(image_prediction.data.reshape(nt, sensor.num_alpha, sensor.num_beta), dims=['t', 'alpha', 'beta'])

movie_pred = eh.movie.Movie(image_plane, times, **obs_args)
obs_pred = movie_pred.observe_same_nonoise(obs_empty, ttype='fast')

# Visualize radiance measurements 
%matplotlib widget

fig, ax = plt.subplots(1,2, figsize=(9.5,4))
image_plane_visualization = xr.DataArray(image_plane, dims=['t', 'alpha', 'beta'])
anim = image_plane_visualization.utils_visualization.animate(ax=ax[0])
obs_pred.plotall('uvdist', 'amp', axis=ax[1], ebar=False)
outpath = 'gifs/{}.prediction.fov{:2.1f}.nspots{:d}.theta_{:1.1f}.phi_{:1.1f}.gif'.format(
    array_name, fov, nspots, np.rad2deg(theta), np.rad2deg(phi))
# anim.save(outpath, writer='imagemagick', fps=10)

Producing clean visibilities from movie with fast FT . . . 


/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/ehtim/obsdata.py:508: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(datalist)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
%matplotlib widget
ax = plt.figure(figsize=(7,7)).add_subplot(projection='3d')
ax.scatter(0,0,0, color='black', s=50)
ax.quiver(0,0,0,*rot_axis, length=0.05, linewidths=3,  label='true')
ax.quiver(0,0,0,*rot_axis_est, length=0.05, linewidths=3, label='estimated', color='r')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend(fontsize=14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …